In [1]:
#@title 0. Google mount

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
#@title 1. 필요 라이브러리 설치
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 # 호환성 문제 해결
!pip install -qU langchain langchain-core langchain-groq langchain-community sentence-transformers chromadb pandas langchainhub python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [3]:
import os
import pandas as pd
from dotenv import load_dotenv
import requests
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
import torch
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

In [4]:
#@title 2. 환경 설정 및 API 키 입력

# 환경 변수 로드
load_dotenv("/content/drive/MyDrive/Colab Notebooks/RAG/.env", override=True)

# Groq API 키 입력
TOUR_API_KEY = os.getenv("TOUR_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [5]:
#@title 3. TourAPI 정보 가져오기


# 분류 코드 매핑
category_map = {
    "A05020100": "한식",
    "A05020200": "서양식",
    "A05020300": "일식",
    "A05020400": "중식",
    "A05020700": "이색음식점",
    "A05020900": "카페/전통찻집"
}

def get_area_based_list():
    url = (
        "http://apis.data.go.kr/B551011/KorService2/areaBasedList2"
        "?numOfRows=20"
        "&pageNo=1"
        "&MobileOS=ETC"
        "&MobileApp=AppTest"
        "&_type=json"
        "&contentTypeId=39"
        "&areaCode=39"
        f"&serviceKey={TOUR_API_KEY}"
    )
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    items = data.get("response", {}).get("body", {}).get("items", {}).get("item", [])
    return items

def get_detail_intro(contentid):
    url = (
        "http://apis.data.go.kr/B551011/KorService2/detailIntro2"
        f"?MobileOS=ETC&MobileApp=AppTest&_type=json&contentId={contentid}&contentTypeId=39"
        f"&numOfRows=1&pageNo=1&serviceKey={TOUR_API_KEY}"
    )
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    item = data.get("response", {}).get("body", {}).get("items", {}).get("item", [{}])[0]
    return item

def get_detail_common(contentid):
    url = (
        "http://apis.data.go.kr/B551011/KorService2/detailCommon2"
        f"?MobileOS=ETC&MobileApp=AppTest&_type=json&contentId={contentid}&numOfRows=1&pageNo=1&serviceKey={TOUR_API_KEY}"
    )
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    item = data.get("response", {}).get("body", {}).get("items", {}).get("item", [{}])[0]
    return item

In [6]:
#@title 4. 데이터 Document 객체로 변환

def create_document(item_basic, item_intro, item_common):
    title = item_basic.get("title", "제목 없음")
    addr = item_basic.get("addr1", "주소 없음")
    cat3_code = item_basic.get("cat3", "")
    category_name = category_map.get(cat3_code, "기타")

    infocenterfood = item_intro.get("infocenterfood", "정보 없음")
    parkingfood = item_intro.get("parkingfood", "정보 없음")
    opentimefood = item_intro.get("opentimefood", "정보 없음")
    firstmenu = item_intro.get("firstmenu", "정보 없음")
    treatmenu = item_intro.get("treatmenu", "정보 없음")
    restdatefood = item_intro.get("restdatefood", "정보 없음")
    mapx = item_basic.get("mapx", "정보 없음")
    mapy = item_basic.get("mapy", "정보 없음")

    overview = item_common.get("overview", "개요 없음")

    page_content = (
        f"가게 이름: {title}\n"
        f"주소: {addr}\n"
        f"분류: {category_name}\n\n"
        f"문의 및 안내: {infocenterfood}\n"
        f"주차 시설: {parkingfood}\n"
        f"영업 시간: {opentimefood}\n"
        f"대표 메뉴: {firstmenu}\n"
        f"취급 메뉴: {treatmenu}\n"
        f"쉬는 날: {restdatefood}\n\n"
        f"개요:\n{overview}"
    )

    metadata = {
        "contentid": item_basic.get("contentid", ""),
        "category": category_name,
        "mapx": item_basic.get("mapx", ""),
        "mapy": item_basic.get("mapy", ""),
        "firstimage": item_basic.get("firstimage", ""),
    }

    return Document(page_content=page_content, metadata=metadata)

In [7]:
items_basic = get_area_based_list()

documents = []
for item in items_basic:
    contentid = item.get("contentid")
    if not contentid:
        continue

    item_intro = get_detail_intro(contentid)
    item_common = get_detail_common(contentid)
    doc = create_document(item, item_intro, item_common)
    documents.append(doc)

print(f"총 {len(documents)}개의 문서 생성")
if documents:
    print("--- 문서 샘플 ---")
    print(documents[0].page_content)
    print("메타데이터:", documents[0].metadata)

총 20개의 문서 생성
--- 문서 샘플 ---
가게 이름: 가는곶 세화
주소: 제주특별자치도 제주시 구좌읍 세화14길 3
분류: 카페/전통찻집

문의 및 안내: 064-782-9006
주차 시설: 불가능
영업 시간: - 화요일~금요일 09:00~19:00<br>- 토요일~월요일 09:00~21:00
대표 메뉴: 흑보리사워도우
취급 메뉴: 에멘탈썬드라이토마토 / 구운제주감자빵 / 쑥보늬밤빵 등
쉬는 날: 연중무휴<br>※ 변동 가능성이 있으므로 전화문의 요망

개요:
가는곶 세화는 제주특별자치도 제주시 구좌읍 세화리에 자리한 감성적인 베이커리로, ‘옆집으로 이사 가고 싶은 세화빵집’을 모토로 운영되고 있다. 이곳은 제주에서 자란 건강한 농작물을 활용한 빵을 만드는 것으로 유명한데, 제주토종흑보리, 대추방울토마토, 푸른콩 등 제주 농부들이 정성껏 키운 재료들을 사용한다. 반죽은 최대한 최소화하여 소화에 부담이 적은 빵을 만드는 것을 지향하고 있다. 시그니처 메뉴로는 에멘탈 치즈와 살짝 말린 토마토가 들어간 ‘에멘탈썬드라이토마토’, 그리고 구운 감자의 풍미가 살아 있는 ‘구운제주감자빵’ 등이 있다. 근처에 세화해수욕장, 평대해변, 비자림 등 자연명소가 가까이 있어 함께 둘러보는 것도 추천한다.
메타데이터: {'contentid': '2850913', 'category': '카페/전통찻집', 'mapx': '126.8606961680', 'mapy': '33.5205279098', 'firstimage': 'http://tong.visitkorea.or.kr/cms/resource/05/2850905_image2_1.jpg'}


In [8]:
#@title 5. 텍스트 분할 (RecursiveCharacterTextSplitter)

if not documents:
    print("문서를 생성해주세요.")
    split_documents = []
else:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        length_function=len,
        is_separator_regex=False,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    split_documents = text_splitter.split_documents(documents)
    print(f"분할 후 총 {len(split_documents)}개의 청크 생성.")
    if split_documents:
        print("\n--- 청크 샘플 ---")
        print(split_documents[0].page_content)

분할 후 총 26개의 청크 생성.

--- 청크 샘플 ---
가게 이름: 가는곶 세화
주소: 제주특별자치도 제주시 구좌읍 세화14길 3
분류: 카페/전통찻집

문의 및 안내: 064-782-9006
주차 시설: 불가능
영업 시간: - 화요일~금요일 09:00~19:00<br>- 토요일~월요일 09:00~21:00
대표 메뉴: 흑보리사워도우
취급 메뉴: 에멘탈썬드라이토마토 / 구운제주감자빵 / 쑥보늬밤빵 등
쉬는 날: 연중무휴<br>※ 변동 가능성이 있으므로 전화문의 요망


In [9]:
#@title 6. 임베딩 모델 설정 (dragonkue/bge-m3-ko)

if not split_documents:
    print("분할된 청크가 없습니다.")
    embedding_model = None
else:
    model_name = "dragonkue/bge-m3-ko"
    model_kwargs = {'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
    encode_kwargs = {'normalize_embeddings': True}

    try:
        embedding_model = HuggingFaceBgeEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
        )
        print(f"임베딩 모델 '{model_name}' 로드 완료 (Device: {model_kwargs['device']}).")

    except Exception as e:
        print(f"임베딩 모델 로드 중 오류 발생: {e}")
        embedding_model = None

/tmp/ipython-input-4256213623.py:12: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

임베딩 모델 'dragonkue/bge-m3-ko' 로드 완료 (Device: cuda).


In [10]:
#@title 7. 벡터 DB 생성 (Chroma) 및 문서 저장

vector_db = None
persist_directory = 'chroma_db_jeju_csv'

if not split_documents or embedding_model is None:
    print("분할된 청크 또는 임베딩 모델이 없습니다.")
else:
    try:
        if os.path.exists(persist_directory) and os.listdir(persist_directory):
            print(f"기존 Chroma DB 로드 중... ({persist_directory})")
            vector_db = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)
            print("기존 Chroma DB 로드 완료.")
        else:
            if os.path.exists(persist_directory):
                print(f"기존 Chroma DB 디렉토리 '{persist_directory}'는 비어있습니다. 새로 생성합니다.")
            else:
                print(f"새로운 Chroma DB 생성 중... ({persist_directory})")

            vector_db = Chroma.from_documents(
                documents=split_documents,
                embedding=embedding_model,
                persist_directory=persist_directory
            )
            vector_db.persist()
            print("새로운 Chroma DB 생성 및 저장 완료.")

    except Exception as e:
        print(f"Chroma DB 처리 중 오류 발생: {e}")
        print("오류로 인해 빈 Chroma DB를 생성하거나, 문서를 새로 추가하려고 시도합니다.")
        if split_documents:
            vector_db = Chroma.from_documents(
                documents=split_documents,
                embedding=embedding_model,
                persist_directory=persist_directory
            )
            vector_db.persist()
            print("오류 복구 시도: Chroma DB 재생성 완료.")
        else:
            print("오류 복구 시도 실패: 추가할 문서가 없습니다.")

새로운 Chroma DB 생성 중... (chroma_db_jeju_csv)
새로운 Chroma DB 생성 및 저장 완료.


/tmp/ipython-input-3223230274.py:25: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


In [11]:
#@title 8. 검색기 설정 (MMR)
retriever = None
if vector_db:
    retriever = vector_db.as_retriever(
        search_type="mmr",
        search_kwargs={'k': 5, 'fetch_k': 20, 'lambda_mult': 0.6}
    )
    print("MMR 검색기 설정 완료.")
else:
    print("벡터 DB가 설정되지 않아 검색기를 생성할 수 없습니다.")

MMR 검색기 설정 완료.


In [12]:
#@title 9. RAG 프롬프트 설정

RAG_PROMPT_TEMPLATE = """
# Role: AI Jeju Travel Planner
You are an expert AI assistant creating personalized Jeju travel itineraries.

## Your Task
Generate a 1-2 day travel itinerary using **ONLY** the locations provided in the [Context] section, based on the user's specific requirements.

## User Requirements
- **Travel Group:** {people}
- **Budget:** Approx. {cost_per_person} KRW per person
- **Food Preferences:** Prefers {food_genre} style meals,
- **Other Conditions:**
  - Vegan: {is_vegan}
  - Allergies: {allergies}
- **Travel Style:** focusing on {travel_preference}.
- **Core Request:** {question}

## Critical Instructions
1.  **Strictly Use Context**: You **MUST** use only the locations from the [Context] below. Do not invent or use any other places.
2.  **Follow Format**: Adhere strictly to the output format example.
3.  **Logical Flow**: Arrange the schedule logically and naturally.
4.  **Justify Choices**: In the one-line review (💬), briefly explain why the place is a good fit for the user's requirements.

## Output Format Example
📅 1일차
🕒 12:00
📍 [장소 이름]
⏰ 영업시간: [영업 정보]
🍽️ 추천메뉴: [메뉴 1, 메뉴 2]
💬 위도와 경도

📅 1일차
🕒 18:00
📍 [장소 이름]
⏰ 영업시간: [영업 정보]
🍽️ 추천메뉴: [메뉴 1, 메뉴 2]
💬 위도와 경도

---
[다음 일정]
---

## Context (Available Locations)
{context}

---

Now, create the itinerary based on all the information above.
"""
rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)
print("RAG 프롬프트 설정 완료.")

RAG 프롬프트 설정 완료.


In [13]:
#@title 10. LLM 설정 (Groq)

llm = None
if GROQ_API_KEY:
    try:
        llm = ChatGroq(
            temperature=0.1,
            model_name="llama-3.1-8b-instant"
        )
        print(f"Groq LLM ({llm.model_name}) 설정 완료.")
    except Exception as e:
        print(f"Groq LLM 설정 중 오류: {e}")
        print("Groq API 키를 확인하거나 네트워크 연결을 확인하세요.")
else:
    print("GROQ_API_KEY가 설정되지 않아 LLM을 초기화할 수 없습니다.")

Groq LLM (llama-3.1-8b-instant) 설정 완료.


In [14]:
#@title 11. RAG 체인 구성 및 실행

rag_chain = None

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 합칩니다."""
    return "\n\n---\n\n".join([d.page_content for d in docs])

if retriever and rag_prompt and llm:
    # RunnablePassthrough.assign을 사용하여 체인을 올바르게 구성합니다.
    # 1. 'question' 키를 사용하여 문서를 검색하고, 그 결과를 'context' 키에 할당합니다.
    # 2. retriever에는 전체 입력이 아닌 'question' 문자열만 전달됩니다.
    # 3. rag_prompt에는 'context'와 함께 원래의 모든 사용자 입력이 전달됩니다.
    rag_chain = (
        RunnablePassthrough.assign(
            context=(itemgetter("question") | retriever | format_docs)
        )
        | rag_prompt
        | llm
        | StrOutputParser()
    )
    print("RAG 체인 구성 완료.")
else:
    print("RAG 체인을 구성하기 위한 요소(retriever, prompt, llm)가 준비되지 않았습니다.")

def ask_question(user_inputs: dict):
    """사용자 입력을 받아 RAG 체인을 실행하고 답변을 출력합니다."""
    if not rag_chain:
        print("RAG 체인이 준비되지 않아 질문을 처리할 수 없습니다. 이전 단계를 확인하세요.")
        return

    print(f"\n질문: {user_inputs.get('question', '[질문 없음]')}")
    try:
        answer = rag_chain.invoke(user_inputs)
        print(f"\n 답변:\n{answer}")
    except Exception as e:
        print(f"질문 처리 중 오류 발생: {e}")

RAG 체인 구성 완료.


In [16]:
#@title 12. 질문하기

if documents and split_documents and vector_db and retriever and llm and rag_chain:
    ask_question({
    "people": "single",
    "cost_per_person": 200000,
    "food_genre": "양식",
    "is_vegan": True,
    "allergies": ["해산물"],
    "travel_preference": ["액티비티", "식도락 여행"],
    "question": "혼자 비싼 음식을 먹으면서 신나게 즐길 수 있는 여행 코스 추천해줘"})

else:
    print("\n데이터 로드 또는 RAG 체인 구성에 실패하여 질문을 실행할 수 없습니다. 위의 셀들을 확인해주세요.")


질문: 혼자 비싼 음식을 먹으면서 신나게 즐길 수 있는 여행 코스 추천해줘

 답변:
📅 1일차
🕒 12:00
📍 거멍국수
⏰ 영업시간: 09:00 ~ 20:00
🍽️ 추천메뉴: 고기국수, 회국수
💬 위도: 33.4443, 경도: 126.5283

📅 1일차
🕒 18:00
📍 경성수산
⏰ 영업시간: 16:00 ~ 22:30
🍽️ 추천메뉴: 모듬회
💬 위도: 33.4443, 경도: 126.5283

---
2일차
---

📅 2일차
🕒 10:00
📍 가는곶 세화
⏰ 영업시간: 09:00 ~ 19:00 (토요일~월요일 09:00~21:00)
🍽️ 추천메뉴: 흑보리사워도우
💬 위도: 33.4443, 경도: 126.5283

📅 2일차
🕒 13:00
📍 고불락
⏰ 영업시간: 화요일~금요일 09:00~19:00, 토요일~월요일 09:00~21:00
🍽️ 추천메뉴: 상추효소밥 정식, 고등어쌈밥
💬 위도: 33.4443, 경도: 126.5283

---
다음 일정
---

📅 2일차
🕒 17:00
📍 제주 갈치왕
⏰ 영업시간: 10:00 ~ 20:00
🍽️ 추천메뉴: 통갈치 해물찜, 통갈치 구이, 가시 없는 갈치조림
💬 위도: 33.4443, 경도: 126.5283
